In [15]:
import torch
import random
import numpy as np
import os
from PIL import Image

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [16]:
'''
Есть 10 классов объектов
Есть ~по 500 изображений каждого класса объектов
Изображения трехканальные(RGB) 100*100 пикселей с вырезанным фоном
Тестовая выборка - по 150-200 изображений тех же классов такого же формата
'''

#загрузка данных

labels = {'AppleGolden' : 0, 'AppleRed' : 1, 'Avocado' : 2, 'Banana' : 3, 'Blueberry' : 4,
          'Cocos' : 5, 'Lemon' : 6, 'Orange' : 7, 'Tomato' : 8, 'Watermelon' : 9}


def load_data(train_test_folder):
    X = []
    Y = []
    for folder in os.listdir(train_test_folder):
        for file in os.listdir(train_test_folder + '/' + folder):
            im = np.asarray(Image.open(train_test_folder + '/' + folder + '/' + file))
            X.append(im)
            Y.append(labels[folder])
    return np.asarray(X),np.asarray(Y)


X_train, Y_train = load_data('Train')
X_test, Y_test = load_data('Test')

In [17]:

X_train = torch.Tensor(X_train)
Y_train = torch.Tensor(Y_train)
Y_train = Y_train.type(torch.LongTensor)
X_test = torch.Tensor(X_test)
Y_test = torch.Tensor(Y_test)
Y_test = Y_test.type(torch.LongTensor)

#ставим каналы 2-ой размерностью для модели
X_train = X_train.permute(0,3,1,2)
X_test = X_test.permute(0,3,1,2)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(torch.Size([5025, 3, 100, 100]),
 torch.Size([5025]),
 torch.Size([1680, 3, 100, 100]),
 torch.Size([1680]))

In [18]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        '''
        Количество слоев и параметры подбирались эмпирически,
        чтобы получить высокую точность и высокую производительность
        '''
        
        #слои свертки + ReLUn активация + Max pooling
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=4, kernel_size=5, padding=2)
        self.act1 = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = torch.nn.Conv2d(in_channels=4,out_channels=8, kernel_size=5, padding=2)
        self.act2 = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = torch.nn.Conv2d(in_channels=8,out_channels=16, kernel_size=5, padding=2)
        self.act3 = torch.nn.ReLU()
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        # полносвязыне слои
        self.fc1 = torch.nn.Linear(12*12*16, 512)
        self.act5 = torch.nn.ReLU()
        
        self.fc2 = torch.nn.Linear(320,160)
        self.act6 = torch.nn.ReLU()
        
        self.fc3 = torch.nn.Linear(160,10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.act3(x)
        x = self.pool3(x)

        # 'вытягивание' тензоа после слоев свертки в матрицу. Строки - объекты. столбцы - полученные прзнаки
        x = x.reshape(x.size(0), x.size(1) * x.size(2) * x.size(3))
        
        #полносвязные слои
        x = self.fc1(x)
        x = self.act5(x)
        x = self.fc2(x)
        x = self.act6(x)
        x = self.fc3(x)
        
        return x
    
classifier = Classifier()

In [19]:
#перенос модели на видеокарту, если есть возможность
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
classifier = classifier.to(device)

In [20]:
#функция потерь для многоклассовой классификации - кросс-энтропия
loss = torch.nn.CrossEntropyLoss()

#оптимизатор выбран эмпирически 
optimizer = torch.optim.Adam(classifier.parameters(), lr=1.0e-3)

In [ ]:
#задаем размер батча
batch_size = 100

test_accuracy_history = []
test_loss_history = []

#перенос на видеокарту для ускорения расчетов, если есть такая возможность
X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)

# цикл обучения
for epoch in range(10000):
    #перемешиваем обучающую выборку
    order = np.random.permutation(len(X_train))
    #цикл по батчам
    for start_index in range(0, len(X_train), batch_size):
        #обнуляем градиенты от предыдущего батча
        optimizer.zero_grad()
        
        #формируем индексы для батчей
        batch_indexes = order[start_index:start_index+batch_size]
        
        #формируем батчи
        X_batch = X_train[batch_indexes].to(device)
        Y_batch = Y_train[batch_indexes].to(device)
        
        #прямая прогонка батча через сеть
        preds = classifier.forward(X_batch) 
        
        #вычисляем ошибку классификации
        loss_value = loss(preds, Y_batch)
        #расчет градиентов для корректировки параметров
        loss_value.backward()
        
        #корректировка параметров сети
        optimizer.step()
    
    #тестируем на тестовых данных
    test_preds = classifier.forward(X_test)
    test_loss_history.append(loss(test_preds, Y_test).data.cpu())
    
    #вычисляем метрику точности классификации    
    accuracy = (test_preds.argmax(dim=1) == Y_test).float().mean().data.cpu()
    test_accuracy_history.append(accuracy)
    
    print('accuracy:', accuracy)

accuracy: tensor(0.9619)
accuracy: tensor(0.9839)
accuracy: tensor(0.9815)
accuracy: tensor(0.9845)
accuracy: tensor(0.9798)
accuracy: tensor(0.9792)
